## torchtextを用いたDataset,DataLoaderの実装

In [3]:
#パッケージのインポート
import glob
import os
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
#import torchvision
#from torchvision import models,transforms
import cv2 
import time
import math
from matplotlib import cm

## 前処理と単語分割の関数を実装

In [4]:
#単語分割はJanome
from janome.tokenizer import Tokenizer

j_t=Tokenizer()

def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text,wakati=True)]

In [5]:
#前処理として正規化する関数を定義
import re

def preprocessing_text(text):
    #半角全角の統一
    #今回は無視

    #英語の小文字化
    #今回は無視
    #output=output.lower()

    #改行、半角スペース、全角スペースを削除
    text=re.sub("\r","",text)
    text=re.sub("\n","",text)
    text=re.sub("　","",text)
    text=re.sub(" ","",text)

    #数字文字の一律0化
    text=re.sub(r"[0-9 ０-９]","0",text) #数字
    #記号と数字の除去
    #今回は無視。半角記号・数字・英字
    #今回は無視。全角記号

    #特定の文字を正規表現で置換する
    #今回は無視

    return text



In [6]:
#前処理とJanomeの単語分割を合わせた関数を定義する

def tokenizer_with_preprocessing(text):
    text=preprocessing_text(text) #前処理の正規化
    ret=tokenizer_janome(text) #Janomeの単語分割
    return ret


In [7]:
#動作確認
text="昨日は　とても暑く、気温が36度もあった。"
print(tokenizer_with_preprocessing(text))

['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


## 文章データの読み込み

In [8]:
import torchtext
from torchtext import data

In [9]:
#tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義する。
#文章とラベルの両方に用意
max_length=25

TEXT=torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
use_vocab=True,lower=True,include_lengths=True, batch_first=True,fix_length=max_length)

LABEL=data.Field(sequential=False,use_vocab=False)

In [10]:
#data.TabularDataset 詳細
#https://torch.text.readthedocs.io/en/latest/examples.html?highlight=data.TabularDataset.splits

#フォルダ「data」から各csvファイルを読み込み、Datasetにします
#1行がTEXTとLABELで区切られていることをfieldsで指示します

train_ds,val_ds,test_ds=torchtext.data.TabularDataset.splits(path="./data/",
train="text_train.tsv",validation="text_val.tsv",test="text_test.tsv", format="tsv",
fields=[("Text",TEXT),("Label",LABEL)])

In [11]:
#動作確認
print("訓練データの数",len(train_ds))
print("1つ目の訓練データ",vars(train_ds[0]))
print("2つ目の訓練データ",vars(train_ds[1]))

訓練データの数 4
1つ目の訓練データ {'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
2つ目の訓練データ {'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}


## 単語の数値化

In [12]:
#ボキャブラリーを作成します
#訓練データtrainの単語からmin_freq以上の頻度の単語を使用してボキャブラリー(単語集)を構築
TEXT.build_vocab(train_ds,min_freq=1)

#訓練データtrainの単語からmin_freq以上の頻度を出力(頻度min_freqよりも大きいものが出力されます)
TEXT.vocab.freqs #出力させる

Counter({'王': 1,
         'と': 5,
         '王子': 1,
         '女王': 1,
         '姫': 1,
         '男性': 1,
         '女性': 1,
         'が': 3,
         'い': 1,
         'まし': 1,
         'た': 1,
         '。': 4,
         '機械': 1,
         '学習': 1,
         '好き': 1,
         'です': 1,
         '本章': 2,
         'から': 1,
         '自然': 1,
         '言語': 1,
         '処理': 1,
         'に': 1,
         '取り組み': 1,
         'ます': 2,
         'で': 1,
         'は': 1,
         '商品': 1,
         'レビュー': 1,
         'の': 4,
         '短い': 1,
         '文章': 4,
         'に対して': 1,
         '、': 3,
         'その': 1,
         'ネガティブ': 1,
         'な': 4,
         '評価': 2,
         'を': 3,
         'し': 3,
         'て': 2,
         'いる': 2,
         'か': 2,
         'ポジティブ': 1,
         '0': 1,
         '値': 1,
         'クラス': 1,
         '分類': 2,
         'する': 1,
         'モデル': 1,
         '構築': 1})

In [13]:
#ボキャブラリーの単語をidに変換した結果を出力
#頻度がmin_freqより小さい場合は未知語<unk>になる

TEXT.vocab.stoi #出力。string to identifiers 文字列をidへ

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fd5d9a96fd0>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
     

## DataLoaderの作成

In [14]:
#DataLoaderを作成します(torchtextの文脈では単純にiteraterと呼ばれています)
train_dl=torchtext.data.Iterator(train_ds,batch_size=2,train=True)

val_dl=torchtext.data.Iterator(val_ds,batch_size=2,train=False,sort=False)

test_dl=torchtext.data.Iterator(test_ds,batch_size=2,train=False,sort=False)

In [15]:
#動作確認 検証データのデータセットで確認
batch=next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[46,  2, 47,  2, 40,  2, 42,  2, 48,  2, 39,  8, 19, 29, 23,  3,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1]]), tensor([16,  6]))
tensor([0, 1])
